<a href="https://colab.research.google.com/github/marialuquin/IA-YOLO/blob/main/pipeline_crockoaches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.efficientnet import EfficientNetB0

from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


# =============================================================================
# setup parameters
# =============================================================================

epoch_=10
batch_size_=32
chanel=3
img_dim=224
batch_size_=16
net = 'vgg16' #vgg19, resnet50, inception_v3, inception_resnet_v2, efficientnet
# =============================================================================
# Load  dataset Crockoaches
# =============================================================================

# Definir el directorio base donde están las carpetas de imágenes
base_dir = '/content/drive/MyDrive/Dataset_Crockoaches/Crockoaches/train/'

# Crear generadores de imágenes con aumentación de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,          # Rotación de hasta 40 grados
    width_shift_range=0.2,      # Desplazamiento horizontal de hasta el 20%
    height_shift_range=0.2,     # Desplazamiento vertical de hasta el 20%
    shear_range=0.2,            # Transformación de corte
    zoom_range=0.2,             # Zoom de hasta el 20%
    horizontal_flip=True,       # Inversión horizontal
    fill_mode='nearest',        # Estrategia de llenado de nuevos píxeles
    brightness_range=[0.8, 1.2],# Variación en el brillo de 80% a 120%
    validation_split=0.2)       # 20% de los datos para validación

test_datagen = ImageDataGenerator(rescale=1./255)

# Cargar las imágenes y separarlas en entrenamiento y validación
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_dim, img_dim),  # Ajusta el tamaño de las imágenes según sea necesario
    batch_size=batch_size_,
    class_mode='categorical',
    subset='training')  # Conjunto de entrenamiento

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_dim, img_dim),
    batch_size=batch_size_,
    class_mode='categorical',
    subset='validation')  # Conjunto de validación

# Crear un generador para el conjunto de prueba
# Asumiendo que tienes un directorio separado para el conjunto de prueba
test_dir = '/content/drive/MyDrive/Dataset_Crockoaches/Crockoaches/test/'
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_dim, img_dim),
    batch_size=batch_size_,
    class_mode='categorical')

print('Classes:', train_generator.class_indices)
print('Training samples:', train_generator.samples)
print('Validation samples:', validation_generator.samples)
print('Testing samples:', test_generator.samples)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
nb_class = len(labels)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size



# =============================================================================
# Fine-Tuning
# =============================================================================
# Cargar el modelo base VGG16 sin las capas superiores
if net == 'vgg16':
  base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_dim, img_dim, chanel))
if net == 'vgg19':
  base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_dim, img_dim, chanel))
if net == 'resnet50':
  base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_dim, img_dim, chanel))
if net == 'inception_v3':
  base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_dim, img_dim, chanel))
if net == 'inception_resnet_v2':
  base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(img_dim, img_dim, chanel))
if net == 'efficientnet':
  base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_dim, img_dim, chanel))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir nuestras propias capas superiores
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.25),
    Dense(nb_class, activation='softmax')  # Capa final con 4 neuronas y activación softmax
])

# =============================================================================
# Metrics evaluation
# =============================================================================
metrics_ = [
    keras.metrics.FalseNegatives(name="fn"),

    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall")
]

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[metrics_, 'accuracy'])



checkpoint = ModelCheckpoint('/content/drive/MyDrive/Dataset_Crockoaches/Modelos/'+net+'_best_model.h5',
                              verbose=1,
                              monitor='val_loss',
                              save_best_only=True,
                              mode='min')

es = EarlyStopping(monitor='val_loss',
                                   patience=3,
                                   verbose=1,
                                   mode='min',
                                   baseline=None,
                                   restore_best_weights=True)


history_model = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=epoch_,
    callbacks=[es, checkpoint],
    verbose=1
)


# Cargar el mejor modelo guardado
model.load_weights('/content/drive/MyDrive/Dataset_Crockoaches/Modelos/'+net+'_best_model.h5')

model.save('/content/drive/MyDrive/Dataset_Crockoaches/Modelos/'+net+'_modelo_finetuned.h5')


plt.plot(history_model.history['precision'])
plt.plot(history_model.history['val_precision'])
plt.title('Model precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('/content/drive/MyDrive/Dataset_Crockoaches/Plots/'+net+'_Precision.png')
plt.clf()

# Plot training & validation loss values.
plt.plot(history_model.history['accuracy'])
plt.plot(history_model.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('/content/drive/MyDrive/Dataset_Crockoaches/Plots/'+net+'_accuracy.png')
plt.clf()

# Evaluar el modelo en el conjunto de validación
evaluation = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)

# Guardar los resultados en un archivo de texto
with open('/content/drive/MyDrive/Dataset_Crockoaches/Resultados/'+net+'_resultados_evaluacion.txt', 'w') as f:
    f.write(f'Pérdida de validación: {evaluation[0]}\n')
    f.write(f'Exactitud de validación: {evaluation[1]}\n')

print(f'Pérdida de validación: {evaluation[0]}')
print(f'Exactitud de validación: {evaluation[1]}')

# Obtener las predicciones del modelo

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes[:len(y_pred)]

# Calcular la exactitud de las predicciones
accuracy = accuracy_score(y_true, y_pred)
print(accuracy)
# Calcular la matriz de confusión
cm = confusion_matrix(y_true, y_pred)

# Visualizar la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('/content/drive/MyDrive/Dataset_Crockoaches/Plots/'+net+'_matriz_de_confusion.png')
plt.show()

# Guardar la matriz de confusión en un archivo de texto
np.savetxt('/content/drive/MyDrive/Dataset_Crockoaches/Resultados/'+net+'_matriz_de_confusion.txt', cm, fmt='%d')

# Reporte de clasificación
report = classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys())
print(report)

# Guardar el reporte de clasificación en un archivo de texto
with open('/content/drive/MyDrive/Dataset_Crockoaches/Resultados/'+net+'_reporte_clasificacion.txt', 'w') as f:
    f.write(report)

Found GPU at: /device:GPU:0
Found 733 images belonging to 4 classes.
Found 182 images belonging to 4 classes.
Found 230 images belonging to 4 classes.
Classes: {'Especie1': 0, 'Especie2': 1, 'Especie3': 2, 'Especie4': 3}
Training samples: 733
Validation samples: 182
Testing samples: 230
58889256/58889256 [==============================] - 1s 0us/step


<ipython-input-2-511b504439bf>:155: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_model = model.fit_generator(


Epoch 1/10
45/45 [==============================] - ETA: 0s - loss: 1.9310 - fn: 309.0000 - tn: 1913.0000 - tp: 408.0000 - precision: 0.6316 - recall: 0.5690 - accuracy: 0.5997
Epoch 1: val_loss improved from inf to 0.33163, saving model to /content/drive/MyDrive/Dataset_Crockoaches/Modelos/vgg16_best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


45/45 [==============================] - 299s 6s/step - loss: 1.9310 - fn: 309.0000 - tn: 1913.0000 - tp: 408.0000 - precision: 0.6316 - recall: 0.5690 - accuracy: 0.5997 - val_loss: 0.3316 - val_fn: 23.0000 - val_tn: 521.0000 - val_tp: 153.0000 - val_precision: 0.9563 - val_recall: 0.8693 - val_accuracy: 0.9205
Epoch 2/10
45/45 [==============================] - ETA: 0s - loss: 0.2767 - fn: 84.0000 - tn: 2099.0000 - tp: 633.0000 - precision: 0.9241 - recall: 0.8828 - accuracy: 0.9066
Epoch 2: val_loss improved from 0.33163 to 0.22209, saving model to /content/drive/MyDrive/Dataset_Crockoaches/Modelos/vgg16_best_model.h5
45/45 [==============================] - 24s 531ms/step - loss: 0.2767 - fn: 84.0000 - tn: 2099.0000 - tp: 633.0000 - precision: 0.9241 - recall: 0.8828 - accuracy: 0.9066 - val_loss: 0.2221 - val_fn: 12.0000 - val_tn: 520.0000 - val_tp: 164.0000 - val_precision: 0.9535 - val_recall: 0.9318 - val_accuracy: 0.9318
Epoch 3/10
45/45 [==============================] - ETA: